<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Fundamentals: Pandas</h1>  

©2000-20 Chris Seddon 
</div>

Execute the following cell to activate styling for this tutorial

In [2]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## Sample: 1
Pandas is a very popular Data Analysis library that is built on top of Numpy.  Pandas excels at two dimensional problems.  

For example consider the file `data/sample.csv`.  I've printed it out below for convenience:
<pre>
Name,   County,         Gender, Age, Height, Weight

Jane,   Berkshire,      F,      30,  178,    68
Peter,  Notts,          M,      27,  170,    59
Silvia, Yorkshire,      F,      42,  183,    66
John,   Dorset,         M,      54,  173,    80
Bill,   Surrey,         M,      31,  169,    70
Ali,    Essex,          M,      30,  181,    75
Zoe,    Cambridgeshire, F,      25,  180,    72
</pre>

This is a comma separated file with 6 fields, a title row and 7 other rows.  We can use Pandas to read this file into memory as a dataframe.  Dataframes are two dimensional data structures that are central to Pandas:

In [3]:
import pandas as pd

df = pd.read_csv("data/sample.csv", engine='python', skipinitialspace=True)
print(type(df))
print(df)

<class 'pandas.core.frame.DataFrame'>
     Name          County Gender  Age  Height  Weight
0    Jane       Berkshire      F   30     178      68
1   Peter           Notts      M   27     170      59
2  Silvia       Yorkshire      F   42     183      66
3    John          Dorset      M   54     173      80
4    Bill          Surrey      M   31     169      70
5     Ali           Essex      M   30     181      75
6     Zoe  Cambridgeshire      F   25     180      72


## Sample: 2
Without the "skipinitialspace=True" parameter, the column headings end up including spaces (this will cause problems identifying columns).

Note that Pandas creates a numerical index column (0 to 6).  All Pandas dataframes have an index column.  The index doesn't have to be an integer; we can choose one of the columns as the index:

In [4]:
import pandas as pd

df = pd.read_csv("data/sample.csv", skipinitialspace=True, index_col=0)
print(type(df))
print(df)

<class 'pandas.core.frame.DataFrame'>
                County Gender  Age  Height  Weight
Name                                              
Jane         Berkshire      F   30     178      68
Peter            Notts      M   27     170      59
Silvia       Yorkshire      F   42     183      66
John            Dorset      M   54     173      80
Bill            Surrey      M   31     169      70
Ali              Essex      M   30     181      75
Zoe     Cambridgeshire      F   25     180      72


## Sample: 3
You'll notice the heading for the index is printed below all the other column headings so that it stands out.  The index values can be used to identify the rows, although it doesn't have to contain unique entries.

To begin with we will print out just the index: 

In [5]:
print(df.index)

Index(['Jane', 'Peter', 'Silvia', 'John', 'Bill', 'Ali', 'Zoe'], dtype='object', name='Name')


## Sample: 4
Next, just the column headings (note that the index is not included):

In [6]:
print(df.columns)

Index(['County', 'Gender', 'Age', 'Height', 'Weight'], dtype='object')


## Sample: 5
Often we want to extract the data from the dataframe and copy it to a two dimensional Numpy array.  

We do this with:

In [ ]:
print(type(df.values))
print(df.values)

## Sample: 6
Now we create a new dataframe with the "County" and "Age" columns.  Note we will still keep the index column.

In [ ]:
df2 = df[['County', 'Age']]
print(type(df2))
print(df2)

## Sample: 7
If we want to create a new dataframe with just one column, we still need to use two sets of [ ] brackets:

In [ ]:
df2 = df[['County']]
print(type(df2))
print(df2)

## Sample: 8
If we only use a single set of [ ] brackets, we get a series rather than a dataframe.  A series is a Pandas one dimensional array (including an index):

In [ ]:
series = df['County']
print(type(series))
print(series)

## Sample: 9
Pandas has two methods of searching for data based on index:

The ".loc" method uses the index of the dataset.  A lot of datasets have an index of integers, but the index can be any data type, often "str".  
The ".iloc" metods works with integers - the postion of the index (starting from 0).

Let's look at some ".loc" examples.  

* Select a single row by index "Peter":

In [ ]:
series = df.loc['Peter']
print(type(series))
print(series)

## Sample: 10
* select an inclusive slice of rows between "Peter" and "Bill"

In [ ]:
df2 = df.loc['Peter':'Bill']
print(type(df2))
print(df2)

## Sample: 11
* select rows "Peter" and "Bill" and columns "County", "Height" and "Weight"

In [ ]:
df2 = df.loc[['Peter', 'Bill'], ['County', 'Height', 'Weight']]
#print(df.loc[['Peter', 'Bill'], ['County', 'Height', 'Weight']])

print(type(df2))
print(df2)

## Sample: 12
* select all rows, but only columns "County" and "Gender"

In [ ]:
df2 = df.loc[:, ['County', 'Gender']]
print(type(df2))
print(df2)

## Sample: 13
The ".iloc" method works as above, except we substitute integers for the index.

* select row 3

In [ ]:
series = df.iloc[3]
print(type(series))
print(series)

## Sample: 14
* select rows 3, 6, 2 and 4

In [ ]:
df2 = df.iloc[[3, 6, 2, 4]]
print(type(df2))
print(df2)

## Sample: 15
* row 3 column 2

In [ ]:
value = df.iloc[3, 2]
print(type(value))
print(value)

## Lerwick : 1
Now we seen hoe Pandas works on a simple file we turn out attention to some real data.  

The file "data/lerwick.txt" contains met office data that is published on their web site.  Let's take a look at it:

In [ ]:
%%bash
cat data/lerwick.txt

## Lerwick : 2
When working with real data, we find some rows have missing data and other rows have extra characters like * and #.  Other rows have an additional column with the string "Provisional":
<pre>
   1982   3    6.7     2.7       1   134.5   108.5*
   2010   1    4.3     0.5      13   113.7    38.5*
   2012   4    7.1     2.5       6    93.3*  108.5#
   2012   5   10.1     4.1       2    71.7*  227.7#
   2015   9   13.3*    9.8*      0*   88.2*   38.5#  Provisional
</pre>

Furthermore, the file is not a "comma seperated value" file.  Nevertheless, Pandas can still read data from this file.  Suprisingly, Pandas still uses "read_csv" even though the file doesn't have any commas.  However we now need to specify a separator to define the field boundaries.

The separator between fields as any combinations of spaces, *s and #s.  We define the separator as a regular expression (regex): <pre>[ *#]+</pre>
Note that if we use a regex as a separator, we have to use the "python" engine (as opposed to the "C" engine which does not support regex, but is faster than the "python" engine).  Additionally, in the above file, we ignore the first 7 lines and rather than using the headings present in the file (awkward because they span two lines) we define out own headings.

We can now read from the file:

In [ ]:
import pandas as pd
pd.set_option('display.precision', 1)
pd.set_option('display.width', None)        # None means all data displayed
pd.set_option('display.max_rows', None)

column_names = ['year', 'month', 'tmax', 'tmin', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment']
lerwick_data = pd.read_csv("data/lerwick.txt", 
                           skiprows = 7,
                           engine = 'python',
                           names = column_names, 
                           skipinitialspace = True, 
                           sep = '[*# ]+')
print(lerwick_data)

## Lerwick : 3
Python provides a few basic methods to get to the feel of the dataframe:  

|   |   |   |
|:---|---|---|
| head(): | print first five rows |
| tail(): | print last five rows |
| sample(5) | print random sample 5 of rows |
| shape: | print number of rows/columns in a tuple |
| describe(): | calculate measures of central tendency |
| info(): | print memory footprint and datatypes |

Let's take a look at the first 5 rows:

In [ ]:
print(lerwick_data.head())

## Lerwick : 4
... and the last 5 rows:

In [ ]:
print(lerwick_data.tail())

## Lerwick : 5
A random sample of 5 rows:

In [ ]:
print(lerwick_data.sample(5))

## Lerwick : 6
The shape of the dataframe:

In [ ]:
print(lerwick_data.shape)

## Lerwick : 7
Various statistics:

In [ ]:
print(lerwick_data.describe())

## Lerwick : 8
Memory footprint etc:

In [ ]:
lerwick_data.info()

## Lerwick : 9
Before we move on to another dataset, one last example with the Lerwick data.  

Lets print out the beginning and end of the dataframe (20 rows in total), sorted by "tmax":

In [ ]:
tmax = lerwick_data.sort_values('tmax', axis=0, ascending=False)
pd.set_option('display.max_rows', 20)
print(tmax)

## Oxford : 1
Now we move on to a more advanced example.  

This time we are using the Met Office data for Oxford; this data is much older and stretches back to 1830: 

In [ ]:
%%bash
head -20 data/oxford.txt
echo "..."
tail -20 data/oxford.txt

## Oxford: 2
Before we look at the details, it would be helpful to run the full example.  We can highlight the important points afterwards.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.precision', 1)
pd.set_option('display.width', 100)
    
def main(): 
    # set column names and read in data from file
    column_names = ['year', 'month', 'tmax', 'tmin', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment']
    oxford_data = pd.read_csv("data/oxford.txt", 
                              engine = 'python', 
                              skiprows = 7, 
                              names = column_names, 
                              na_values = ['---'], 
                              skipinitialspace = True, 
                              sep = '[*# ]+')

    # some of the tmin values are missing, so drop these rows
    oxford_data.dropna()
    
    # create a new column from year and month columns
    oxford_data['period'] = oxford_data.apply(
        lambda row : (row.year//4)*4, raw = True, 
        axis = 1
        )

    # drop columns we are not using (not necessary)
    oxford_data.drop(['year', 'month', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment'], axis = 1, inplace = True)

    # group results into 4 year periods
    # the new column (period) becomes the index
    summary = oxford_data.groupby(['period']).aggregate(np.mean)

    # plot the data
    ax = summary.plot(figsize=(10, 6), 
                               title = 'Oxford : Average Min and Max Temperatures (over 4 year periods)', 
                               # x defaults to index
                               y = ['tmin', 'tmax'], 
                               color = ['red', 'green'], 
                               kind = 'bar')

    ax.set_xlabel("4 year period")
    ax.set_ylabel(f"{chr(0x2103)}")     # degrees C
    for item in [ax. title, ax.xaxis.label, ax.yaxis.label]:
        item.set_fontsize(20)
    plt.show()
main()

## Oxford : 2
There are two key lines in the above example.  
The first is where we create a new column: 

<pre>
oxford_data['period'] = oxford_data.apply(
        lambda row : (row.year//4)*4, 
        raw = True, 
        axis = 1
        )
</pre>

The "apply" method is used to apply a function along an axis of the DataFrame, often to create a new row or column.  In this case we are creating new column for each row (axis=1).  

The "apply" method takes a lambda to calculate the new values for the column:<pre> lambda row : (row.year//4)*4</pre>
The lambda works on each row, taking the year and rounding it to the nearest 4 year period.  By the way, the "row" input can be supplied as a series (row=False) or as a Numpy array (row=True).

In [ ]:
column_names = ['year', 'month', 'tmax', 'tmin', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment']
oxford_data = pd.read_csv("data/oxford.txt", 
                          engine = 'python', 
                          skiprows = 7, 
                          names = column_names, 
                          na_values = ['---'], 
                          skipinitialspace = True, 
                          sep = '[*# ]+')

# some of the tmin values are missing, so drop these rows
oxford_data.dropna()
    
oxford_data['period'] = oxford_data.apply(
        lambda row : (row.year//4)*4, raw = True, 
        axis = 1
        )
print(oxford_data[750:760])
print(oxford_data[846:856])

## Oxford : 3
There is far too much data to plot month by month, so we have aggregated the "min" and "max" average temperatures over 48 month periods.  That's why we created the "period" column.

The second key line is where we aggregate the data into groups of 48 (rows with the same value of "period").  In this example we create a new dataframe with the "np.mean" of each of the 48 rows.

<pre>
summary = oxford_data.groupby(['period']).aggregate(np.mean)
</pre>

In [ ]:
# drop columns we are not using (not necessary)
oxford_data.drop(['year', 'month', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment'], axis = 1, inplace = True)


# group results into 4 year periods
# the groupby column (period) becomes the index
summary = oxford_data.groupby(['period']).aggregate(np.mean)
print(summary)

## Olympics: 1
Our new example shows how to extract data from a dataframe based on conditions.  

We will display all countries who were awarded more golds than South Korea.  Let's look at the data:

In [ ]:
%%bash
head -25 data/olympics_2012_medal_table.txt
echo "..."

## Olympics: 2
Here is the full example:

In [ ]:
import pandas as pd
import pylab as pl
pd.set_option('display.width', 100)

# read in medal table (n.b. delimiters contain at least 2 spaces and sometimes a bracket) 
medal_table = pd.read_csv("data/olympics_2012_medal_table.txt",
                           engine = 'python',
                           skiprows = 1,
                           sep = '[ )(]{2,}')

korean_golds = medal_table[medal_table.Id == "KOR"]["Gold"].values[0]
print("Korea earned {} golds".format(korean_golds))
print("\nCountries with more golds than South Korea:")
result = medal_table[medal_table.Gold > korean_golds][["Country", "Gold"]]
print(result.to_string(index=False))

## Olympics: 3
In the above example our first problem is to determine the field delimeter.  A careful look at the data reveals spaces in the Country field, but there are never more than 1 consecutive space.  If we include space and ( ) brackets as delimiter characters we notice that every field has at least 2 delimeter characters.  One of the trickest rows is: <pre>
19   Czech Republic (CZE)     4     3      3    10
</pre>
Here the "Country" and "Id" columns are separated by " (".  This is the minimum 2 character separator.  

So we can define the separator as the regex with the three delimeter characters space and the two brackets ( ) as:<pre>[ )(] {2,}</pre>
where `{2,}` means 2 or more.

In [ ]:
print(medal_table)

## Olympics: 4
To determine how many golds were awarded to South Korea, we extract a series.  Unfortunately the series include (as always) the index.  What we need to do is convert the series to a Numpy array and extract the first element of that array:

In [ ]:
korean_golds = medal_table[medal_table.Id == "KOR"]["Gold"]
print(type(korean_golds))
print(f"{korean_golds}\n")

korean_golds = medal_table[medal_table.Id == "KOR"]["Gold"].values
print(type(korean_golds))
print(f"{korean_golds}\n")

korean_golds = medal_table[medal_table.Id == "KOR"]["Gold"].values[0]
print(type(korean_golds))
print(f"{korean_golds}\n")

## Excel: 1
Pandas makes it easy to read and write dataframes from/to Excel spreadshhets.  Other Python modules can also be used:
<a href="https://openpyxl.readthedocs.io/en/stable/">openpyxl</a>

Here is the "write" example:

In [ ]:
import pandas as pd
import pylab as pl
pd.set_option('display.precision', 1)
pd.set_option('display.width', 100)

def main(): 
    column_names = ['year', 'month', 'tmax', 'tmin', 'air-frost-days', 'rain(mm)', 'sun(hours)', 'comment']
    lerwick_data = pd.read_csv("data/lerwick.txt", 
                               skiprows = 8, 
                               names = column_names, 
                               skipinitialspace = True,
                               engine = 'python', 
                               sep = '[*# ]+')
    lerwick_data.to_excel('data/lerwick.xlsx', index = False)
main()

## Excel: 2
Here is the "read" example:

In [ ]:
import pandas as pd
import pylab as pl
pd.set_option('display.precision', 1)
pd.set_option('display.width', 100)

def main(): 
    lerwick_data_df = pd.read_excel('data/lerwick.xlsx', 'Sheet1')
    print(lerwick_data_df)

main()